In [6]:
import pandas as pd
import os
from pathlib import Path
import re

In [7]:
excel_dir = Path("/Users/rutmehta/Developer/NJBDA/course_equiv analysis/excels")

In [10]:
def extract_data_from_excel(file_path):
    # Read the Excel file
    try:
        # Try with default sheet (0)
        df = pd.read_excel(file_path)
        
        # Look for required columns (case-insensitive search)
        required_cols = ['SI', 'Course ID', 'Course Title', 'RI', "EQ 'FN'"]
        found_cols = []
        
        # Create a mapping between required column names and actual column names in the file
        col_mapping = {}
        for req_col in required_cols:
            for col in df.columns:
                # Check if column name contains the required name (case-insensitive)
                if req_col.lower() in col.lower():
                    col_mapping[req_col] = col
                    found_cols.append(col)
                    break
        
        # Check if all required columns were found
        if len(found_cols) < len(required_cols):
            missing = set(required_cols) - set(col_mapping.keys())
            print(f"Warning: Couldn't find these columns in {file_path.name}: {missing}")
            print(f"Available columns: {df.columns.tolist()}")
            
        # Select only the required columns that were found
        if found_cols:
            df_selected = df[found_cols]
            return df_selected
        else:
            print(f"No required columns found in {file_path.name}")
            return pd.DataFrame()
            
    except Exception as e:
        print(f"Error processing {file_path.name}: {str(e)}")
        return pd.DataFrame()

In [11]:
all_dataframes = {}
    
# Iterate through all Excel files in the directory
excel_files = list(excel_dir.glob("*.xlsx")) + list(excel_dir.glob("*.xls"))

if not excel_files:
    print(f"No Excel files found in {excel_dir}")

print(f"Found {len(excel_files)} Excel files to process.")

# Process each Excel file
for file_path in excel_files:
    file_name = file_path.name
    print(f"Processing {file_name}...")
    
    # Extract data from the file
    df = extract_data_from_excel(file_path)
    
    # Store DataFrame in dictionary with filename as key
    if not df.empty:
        all_dataframes[file_name] = df
        print(f"  Extracted {len(df)} rows with {len(df.columns)} columns")
    else:
        print(f"  No data extracted from {file_name}")

# Example of how to access each DataFrame
print("\nSummary of extracted DataFrames:")
for file_name, df in all_dataframes.items():
    print(f"{file_name}: {df.shape[0]} rows, {df.shape[1]} columns")
    # Display first few rows of each DataFrame
    if not df.empty:
        print(df.head(3))
        print()

Found 10 Excel files to process.
Processing (2025.03.19) Centenary Master Evaluation Matrix (Latest SI, Latest RI).xlsx...
  Extracted 24337 rows with 5 columns
Processing (2025.03.19) Kean Master Evaluation Matrix (Latest SI, Latest RI).xlsx...
  Extracted 24360 rows with 5 columns
Processing (2025.03.19) FDU-Metropolitan Master Evaluation Matrix (Latest SI, Latest RI).xlsx...
  Extracted 24290 rows with 5 columns
Processing (2025.03.19) Montclair Master Evaluation Matrix (Latest SI, Latest RI).xlsx...
  Extracted 24347 rows with 5 columns
Processing (2025.03.19) NJCU Master Evaluation Matrix (Latest SI, Latest RI).xlsx...
  Extracted 24360 rows with 5 columns
Processing (2025.03.19) Rowan Master Evaluation Matrix (Latest SI, Latest RI).xlsx...
  Extracted 24341 rows with 5 columns
Processing (2025.03.19) Rider Master Evaluation Matrix (Latest SI, Latest RI).xlsx...
  Extracted 24358 rows with 5 columns
Processing (2025.03.19) FDU-Florham Master Evaluation Matrix (Latest SI, Latest RI

In [23]:
# Define regex patterns for CS, Math, and Data Science courses
cs_pattern = r'^(CS|COMP|CMPS|CSCI|CPE|CIS|IT|CSE|CPS|INF)\d'  # Computer Science
math_pattern = r'^(MATH|MTH|MAT|MA)\d'  # Mathematics
data_pattern = r'^(DATA|DS|DSC|STAT|STA|DSCI)\d'  # Data Science

# Filter each dataframe to keep only CS, Math, and Data Science courses
filtered_dataframes = {}

for file_name, df in all_dataframes.items():
    # Get the Course ID column name (it might be slightly different in some files)
    course_id_col = [col for col in df.columns if 'Course ID' in col][0]
    
    # Create a combined filter for CS, Math, and Data Science courses
    cs_filter = df[course_id_col].str.match(cs_pattern, case=False)
    math_filter = df[course_id_col].str.match(math_pattern, case=False)
    data_filter = df[course_id_col].str.match(data_pattern, case=False)
    
    # Combine all filters with logical OR
    combined_filter = cs_filter | math_filter | data_filter
    
    # Apply the filter to keep only CS, Math, and Data Science courses
    filtered_df = df[combined_filter].copy()
    
    # Store the filtered dataframe
    filtered_dataframes[file_name] = filtered_df
    
    # Print summary of filtering for this institution
    print(f"{file_name}:")
    print(f"  Original: {len(df)} courses")
    print(f"  Filtered: {len(filtered_df)} courses")
    print(f"  Courses removed: {len(df) - len(filtered_df)} courses")
    
    # Count courses by discipline
    cs_count = sum(filtered_df[course_id_col].str.match(cs_pattern, case=False))
    math_count = sum(filtered_df[course_id_col].str.match(math_pattern, case=False))
    data_count = sum(filtered_df[course_id_col].str.match(data_pattern, case=False))
    
    print(f"  CS courses: {cs_count}")
    print(f"  Math courses: {math_count}")
    print(f"  Data Science courses: {data_count}")


(2025.03.19) Centenary Master Evaluation Matrix (Latest SI, Latest RI).xlsx:
  Original: 24337 courses
  Filtered: 1537 courses
  Courses removed: 22800 courses
  CS courses: 759
  Math courses: 768
  Data Science courses: 10
(2025.03.19) Kean Master Evaluation Matrix (Latest SI, Latest RI).xlsx:
  Original: 24360 courses
  Filtered: 1538 courses
  Courses removed: 22822 courses
  CS courses: 759
  Math courses: 769
  Data Science courses: 10
(2025.03.19) FDU-Metropolitan Master Evaluation Matrix (Latest SI, Latest RI).xlsx:
  Original: 24290 courses
  Filtered: 1538 courses
  Courses removed: 22752 courses
  CS courses: 759
  Math courses: 769
  Data Science courses: 10
(2025.03.19) Montclair Master Evaluation Matrix (Latest SI, Latest RI).xlsx:
  Original: 24347 courses
  Filtered: 1536 courses
  Courses removed: 22811 courses
  CS courses: 759
  Math courses: 767
  Data Science courses: 10
(2025.03.19) NJCU Master Evaluation Matrix (Latest SI, Latest RI).xlsx:
  Original: 24360 cour

In [25]:
# Display a sample from each filtered dataframe
print("\nSamples from filtered dataframes:")
for file_name, df in filtered_dataframes.items():
    print(f"{file_name} (first 3 rows of {len(df)} total):")
    if not df.empty:
        print(df.head(3))
    else:
        print("  No matching courses found")
    print()


Samples from filtered dataframes:
(2025.03.19) Centenary Master Evaluation Matrix (Latest SI, Latest RI).xlsx (first 3 rows of 1537 total):
     SI Course ID                             Course Title  RI EQ 'FN'
686  AT   MATH070                       MATH SKILLS REVIEW  CE     NaN
687  AT   MATH071                        FOUNDATIONAL MATH  CE     NaN
688  AT   MATH073  INTRODUCTION TO ALGEBRA I - PRE-ALGEBRA  CE     NaN

(2025.03.19) Kean Master Evaluation Matrix (Latest SI, Latest RI).xlsx (first 3 rows of 1538 total):
     SI Course ID                             Course Title  RI  \
688  AT   MATH070                       MATH SKILLS REVIEW  KE   
689  AT   MATH071                        FOUNDATIONAL MATH  KE   
690  AT   MATH073  INTRODUCTION TO ALGEBRA I - PRE-ALGEBRA  KE   

              EQ 'FN'  
688               NaN  
689  MATH0901 'K1,K3'  
690               NaN  

(2025.03.19) FDU-Metropolitan Master Evaluation Matrix (Latest SI, Latest RI).xlsx (first 3 rows of 1538 total)

In [31]:
# Create a combined dataframe of non-transferring courses
non_transferring_courses = []

for file_name, df in filtered_dataframes.items():
    # Get the EQ column name (it might be slightly different in some files)
    eq_col = [col for col in df.columns if "EQ 'FN'" in col][0]
    
    # Get the RI column name
    ri_col = [col for col in df.columns if 'RI' == col][0]
    
    # Filter for courses with NaN in the EQ column
    non_transferring = df[df[eq_col].isna()].copy()
    
    # Add to our list of dataframes
    non_transferring_courses.append(non_transferring)

# Combine all the non-transferring courses into a single dataframe
if non_transferring_courses:
    combined_non_transferring = pd.concat(non_transferring_courses, ignore_index=True)
    
    # Count by Receiving Institution (RI)
    ri_counts = combined_non_transferring['RI'].value_counts()
    print("\nCounts by Receiving Institution (RI):")
    for ri, count in ri_counts.items():
        print(f"  {ri}: {count} courses")
    
    # Display the first few rows
    print("\nSample of combined non-transferring courses:")
    print(combined_non_transferring.head())
    
    combined_non_transferring = combined_non_transferring.sort_values(by = ['SI', "EQ 'FN'"])
    # Optional: save to CSV
    combined_non_transferring.to_csv("non_transferring_cs_math_data_courses.csv", index=False)
else:
    print("No non-transferring courses found.")


Counts by Receiving Institution (RI):
  NI: 1101 courses
  RI: 580 courses
  MO: 430 courses
  NU: 398 courses
  RM: 329 courses
  RO: 300 courses
  KE: 268 courses
  FD: 213 courses
  FM: 202 courses
  CE: 180 courses

Sample of combined non-transferring courses:
   SI Course ID                             Course Title  RI EQ 'FN'
0  AT   MATH070                       MATH SKILLS REVIEW  CE     NaN
1  AT   MATH071                        FOUNDATIONAL MATH  CE     NaN
2  AT   MATH073  INTRODUCTION TO ALGEBRA I - PRE-ALGEBRA  CE     NaN
3  AT   MATH074               INTRODUCTION TO ALGEBRA II  CE     NaN
4  AT   MATH080                       ELEMENTARY ALGEBRA  CE     NaN


In [33]:
# Group by SI, Course ID, and Course Title to analyze the pattern
course_analysis = combined_non_transferring.groupby(['SI', 'Course ID', 'Course Title']).agg(
    count=('RI', 'count'),  # Count of receiving institutions
    non_transferring_institutions=('RI', lambda x: list(sorted(x)))  # List of RIs where course doesn't transfer
).reset_index()

# Sort by SI and then by count (descending)
course_analysis = course_analysis.sort_values(by=['SI', 'count'], ascending=[True, False])

# Rename the columns for clarity
course_analysis = course_analysis.rename(columns={
    'count': 'Number_of_Non_Transfers',
    'non_transferring_institutions': 'Non_Transferring_To'
})

# Display the results
print("Analysis of Non-Transferring Courses:")
print(f"Total unique courses: {len(course_analysis)}")
print("\nSample of the analysis (first 10 rows):")
print(course_analysis.head(10))

# Optional: Calculate additional statistics
si_summary = course_analysis.groupby('SI').agg(
    unique_courses=('Course ID', 'count'),
    avg_non_transfers=('Number_of_Non_Transfers', 'mean'),
    max_non_transfers=('Number_of_Non_Transfers', 'max')
).reset_index()

print("\nSummary by Sending Institution (SI):")
print(si_summary)

# Optional: save to CSV
course_analysis.to_csv("non_transferring_course_analysis.csv", index=False)

Analysis of Non-Transferring Courses:
Total unique courses: 1211

Sample of the analysis (first 10 rows):
   SI Course ID                             Course Title  \
0  AT   MATH070                       MATH SKILLS REVIEW   
2  AT   MATH073  INTRODUCTION TO ALGEBRA I - PRE-ALGEBRA   
3  AT   MATH074               INTRODUCTION TO ALGEBRA II   
4  AT   MATH080                       ELEMENTARY ALGEBRA   
1  AT   MATH071                        FOUNDATIONAL MATH   
5  AT   MATH099           ACCELERATED ELEMENTARY ALGEBRA   
6  AT   MATH119                     INTERMEDIATE ALGEBRA   
9  AT   MATH210                               STATISTICS   
7  AT   MATH121              APPLICATIONS OF MATHEMATICS   
8  AT   MATH128                             TRIGONOMETRY   

   Number_of_Non_Transfers                       Non_Transferring_To  
0                       10  [CE, FD, FM, KE, MO, NI, NU, RI, RM, RO]  
2                        9      [CE, FD, FM, KE, MO, NI, NU, RI, RO]  
3                   